<a href="https://colab.research.google.com/github/kevinworthington/geospatial_colab/blob/main/Landsat_Merge_Reduce_Multiple_GeoTiffs_by_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following notebook searches for remote sensing data over a period of time and merges these images together by averaging the values of each of the pixels.

The preset boundary box is editable allowing you to choose where you'd like to do your analysis.

Referenced From: https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

We'll start by installing the needed libraries, but if you have these already, skip to loading the libraries.

In [2]:
%%capture
!pip install leafmap

In [1]:
%%capture
!pip install widgetsnbextension

In [43]:
# Now load the required libraries, each includes their intended purpose in this project
from pystac_client import Client # To query STAC API endpoint
import planetary_computer # Microsoft Data Catalog library
from shapely.geometry import shape,GeometryCollection,box # To work with drawn geometry
from shapely.geometry.polygon import Polygon
import leafmap # A feature-rich interactive map allow us to save drawn geometry (among other things)
import geojson # To parse geospatial data
import folium # Another interactive map
from shapely.ops import transform # The shapely transform module
import pyproj # A reprojection library
import rioxarray # To open raster images
from rioxarray import merge # To merge raster images
import numpy as np # To math raster images

In [44]:
# Create a polygon to define our Area of Interest (AOI), this can be updated within our interactive map
drawn_box: dict = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -106.05059,
              39.856251
            ],
            [
              -106.05059,
              40.443559
            ],
            [
              -105.182217,
              40.443559
            ],
            [
              -105.182217,
              39.856251
            ],
            [
              -106.05059,
              39.856251
            ]
          ]
        ]
      }
    }
  ]
}

In [45]:
# Create an interactive map allowing us to edit our initial AOI
# You can delete the existing polygon and/or draw mulitple polygons which will later be combined to create a boundary box used to search for data
m = leafmap.Map(center=[40, -100], zoom=4)
m.edit_vector(drawn_box)
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [46]:
# save the drawn polygons
m.save_draw_features("data.geojson")

In [47]:
# open the saved polygons. This is the only way to avoid copying and pasting the updated geojson
with open('data.geojson', 'r') as file:
    geojson_data = geojson.load(file)

In [48]:
# Create a boundary box for all the drawn polygons

# Ref: rom https://gis.stackexchange.com/questions/270297/getting-bounding-boxes-for-all-polygons-in-geojson-feature-collection

# Define a list to store the geometry
polygons=[]
for f in geojson_data["features"]:
    polygon: Polygon = shape(f["geometry"])
    polygons.append(polygon)

# Create a colletion with all the geometries
polygons_gc=GeometryCollection(polygons)

# Store the bounds in a new variable
polygons_bbox= box(*polygons_gc.bounds)


In [49]:
# Create a new interactive map with a boundary around all the shapes drawn earlier

# Create the map
folium_map = folium.Map([polygons_bbox.centroid.y,polygons_bbox.centroid.x], zoom_start=10)

# Add drawn boundaries
folium.GeoJson(geojson_data).add_to(folium_map)

# # Add max bounds
folium.GeoJson(polygons_bbox,
    style_function=lambda feature: {
        "fillColor": "#ff0000",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }).add_to(folium_map)

folium_map

In [50]:
# Connect to the STAC endpoint used to find satellite data
cmr_api_url =  "https://planetarycomputer.microsoft.com/api/stac/v1"
client = Client.open(cmr_api_url, modifier=planetary_computer.sign_inplace,)

# Run search
search = client.search(
    collections=["landsat-c2-l2"],
    bbox=polygons_bbox.bounds,
    datetime="2021-03-01/2021-03-30",
)
# Retrieve search results
items = search.item_collection()
print(len(items))

9


In [51]:
# Inspect the first item
items[0]

<Item id=LC08_L2SP_033033_20210324_02_T1>

In [52]:
# visualize the first item
m = leafmap.Map(center=[polygons_bbox.centroid.y,polygons_bbox.centroid.x])
m.add_cog_layer(items[0].assets["nir08"].href, name="")


m

Map(center=[40.2295795, -105.5352155], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [53]:
# Sort the items by cloud_cover
items = search.item_collection()

items_sorted = sorted(items, key=lambda x: x.properties["eo:cloud_cover"])

In [54]:
# Lets short list the items which we'll merge later and then clip
# Start by gathering the hrefs
item_hrefs=[]

for i in items_sorted[0:4]:
    item_hrefs.append(i.assets["nir08"].href)

In [55]:
items[0].properties

{'gsd': 30,
 'created': '2022-05-06T18:07:26.417023Z',
 'sci:doi': '10.5066/P9OGBGM6',
 'datetime': '2021-03-24T17:37:30.160377Z',
 'platform': 'landsat-8',
 'proj:shape': [7841, 7711],
 'description': 'Landsat Collection 2 Level-2',
 'instruments': ['oli', 'tirs'],
 'eo:cloud_cover': 83.11,
 'proj:transform': [30.0, 0.0, 411585.0, 0.0, -30.0, 4423215.0],
 'view:off_nadir': 0,
 'landsat:wrs_row': '033',
 'landsat:scene_id': 'LC80330332021083LGN00',
 'landsat:wrs_path': '033',
 'landsat:wrs_type': '2',
 'view:sun_azimuth': 146.38573471,
 'landsat:correction': 'L2SP',
 'view:sun_elevation': 47.79202879,
 'landsat:cloud_cover_land': 83.11,
 'landsat:collection_number': '02',
 'landsat:collection_category': 'T1',
 'proj:code': 'EPSG:32613'}

In [56]:
# View the map file 'boundary'
# Use the file's lat and long column to center the map

folium_map = folium.Map([polygons_bbox.centroid.y,polygons_bbox.centroid.x], zoom_start = 10)

# Add the boundary
folium.GeoJson(polygons_bbox).add_to(folium_map)

for id, i in enumerate(item_hrefs):
      raster = rioxarray.open_rasterio(item_hrefs[id])
      # Create boundary boxes
      # use teh first item to create a reprojection transformer
      project = pyproj.Transformer.from_crs(items[0].properties["proj:code"], 4326, always_xy=True).transform

      bbox = box(*raster.rio.bounds())
      bbox_transformed = transform(project, bbox)

      folium.GeoJson(bbox_transformed,
          style_function=lambda feature: {
          "color": "purple",
      }).add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

# Merge the datasets

In [60]:
# Start by creating a list of the images
nir_rasters=[]
for i in item_hrefs:
    nir_rasters.append(rioxarray.open_rasterio(i, masked=True))


In [61]:
# Set our boundry to the CRS of the raster so we can use it to clip
project_from_4326 = pyproj.Transformer.from_crs(4326,items[0].properties["proj:code"], always_xy=True).transform
polygons_bbox_transformed = transform(project_from_4326, polygons_bbox)

In [62]:
help (merge.merge_arrays)

Help on function merge_arrays in module rioxarray.merge:

merge_arrays(dataarrays: collections.abc.Sequence[xarray.core.dataarray.DataArray], *, bounds: Optional[tuple] = None, res: Optional[tuple] = None, nodata: Optional[float] = None, precision: Optional[float] = None, method: Union[str, Callable, NoneType] = None, crs: Optional[rasterio.crs.CRS] = None, parse_coordinates: bool = True) -> xarray.core.dataarray.DataArray
    Merge data arrays geospatially.
    
    Uses :func:`rasterio.merge.merge`
    
    .. versionadded:: 0.2 crs
    
    Parameters
    ----------
    dataarrays: list[xarray.DataArray]
        List of multiple xarray.DataArray with all geo attributes.
        The first one is assumed to have the same
        CRS, dtype, and dimensions as the others in the array.
    bounds: tuple, optional
        Bounds of the output image (left, bottom, right, top).
        If not set, bounds are determined from bounds of input DataArrays.
    res: tuple, optional
        Output

In [63]:
# create a custom merge function allowing us to average the overlapping pixels and merger
def custom_merge_avg(old_data, new_data, old_nodata, new_nodata, index=None, roff=None, coff=None):
    old_data[:] = np.nanmean( np.array([ old_data, new_data ]), axis=0 )

nir_merged = merge.merge_arrays(nir_rasters,bounds=polygons_bbox_transformed.bounds,method = custom_merge_avg)

In [64]:
nir_merged

<xarray.DataArray (band: 1, y: 2780, x: 2940)> Size: 33MB
array([[[18292.5  , 17552.   , 15417.   , ..., 19267.   , 19452.   ,
          7870.5  ],
        [18385.   , 18108.   , 17366.5  , ..., 19443.375, 19256.75 ,
          7870.5  ],
        [18292.5  , 18385.   , 18200.   , ..., 20186.   , 19660.75 ,
          7684.   ],
        ...,
        [ 8081.25 ,  7760.5  ,  6889.25 , ..., 22580.   , 21468.188,
          4865.75 ],
        [ 7714.75 ,  6889.25 ,  5647.   , ..., 22230.562, 21244.   ,
          4865.75 ],
        [14971.5  , 13135.   , 10925.   , ..., 23204.375, 23323.625,
          9731.5  ]]], dtype=float32)
Coordinates:
  * x            (x) float64 24kB 4.101e+05 4.102e+05 ... 4.983e+05 4.983e+05
  * y            (y) float64 22kB 4.495e+06 4.495e+06 ... 4.412e+06 4.412e+06
    spatial_ref  int64 8B 0
  * band         (band) int64 8B 1
Attributes:
    AREA_OR_POINT:  Area

In [76]:
nir_merged.rio.crs

CRS.from_epsg(32613)

In [79]:
# save our merged geotiff
nir_merged.to_netcdf("output_data/nir_merged.nc")

In [80]:
satellite = leafmap.download_file("output_data/nir_merged.nc","output_data/nir_merged.nc")
m = leafmap.Map()
m.add_raster(satellite)
m

output_data/nir_merged.nc already exists. Skip downloading. Set overwrite=True to overwrite.


Map(center=[40.229613, -105.54083750000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…